## Tools

An agent can use tools to interact with the world. Tools include functions, other chains, or even other agents. For example, a function connected to Google Search API can be used by an agent to look for up-to-date information beyond its training dataset.

In this notebook, we will explore some built-in tools, then learn how to create custom tools.

## How to access built-in tools?

In [1]:
from langchain.agents import load_tools

tool_names = ["arxiv", "wikipedia"]
tools = load_tools(tool_names)
tools[0]

ArxivQueryRun(name='arxiv', description='A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.arxiv.ArxivError'>, <class 'arxiv.arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000, arxiv_result=<class 'arxiv.arxiv.Result'>))

Some tools require you to provide a base LLM.

In [2]:
tool_names = ["llm-math"]
tools = load_tools(tool_names)  # Error, requires value for `llm` param
tools[0]

ValueError: Tool llm-math requires an LLM to be provided

In [3]:
from langchain.chat_models import ChatOpenAI

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=ChatOpenAI(temperature=0))
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

Other tools require initialization by creating `Tool` object.

In [4]:
from langchain.tools import Tool, DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Useful for when you need to do a search on the internet to find information that another tool can't find. Be specific with your input.",
    ),
]

tools[0].name, tools[0].description

('DuckDuckGo Search',
 "Useful for when you need to do a search on the internet to find information that another tool can't find. Be specific with your input.")

## How to use a tool?

I assume that you know how to create an agent. You can refer to this [webpage](https://python.langchain.com/docs/modules/agents/) for more details.

Provide a tool for an agent by passing in the list of tools during its initialization.

In [5]:
from langchain.agents import AgentType, initialize_agent

agent_executor = initialize_agent(
    tools=load_tools(["wikipedia"]),
    llm=ChatOpenAI(temperature=0),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [6]:
agent_executor.run("How many regions are in the Philippines?")



> Entering new AgentExecutor chain...
I should use Wikipedia to find the answer to this question.
Action: Wikipedia
Action Input: "Regions of the Philippines"
Observation: Page: Regions of the Philippines
Summary: In the Philippines, regions (Filipino: rehiyon; ISO 3166-2:PH) are administrative divisions that primarily serve to coordinate planning and organize national government services across multiple local government units (LGUs). Most national government offices provide services through their regional branches instead of having direct provincial or city offices. Regional offices are usually but not necessarily located in the city designated as the regional center.
As of 2019, the Philippines is divided into 17 regions. 16 of these are mere administrative groupings, each provided by the president of the Philippines with a regional development council (RDC) – in the case of the National Capital Region (Metro Manila), an additional metropolitan authority serves as the coordinating 

'There are 17 regions in the Philippines.'

Let's create an agent with access to some tools and a memory.

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

agent_executor = initialize_agent(
    tools=load_tools(["wikipedia"]),
    llm=ChatOpenAI(temperature=0),
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

In [8]:
agent_executor.run("What is sourdough bread?")



> Entering new AgentExecutor chain...
{
    "action": "Wikipedia",
    "action_input": "sourdough bread"
}
Observation: Page: Sourdough
Summary: Sourdough or sourdough bread is a bread made by the fermentation of dough using wild lactobacillaceae and yeast. Lactic acid from fermentation imparts a sour taste and improves keeping qualities.

Page: List of sourdough breads
Summary: This is a list of sourdough breads. Sourdough is prepared through the fermentation of dough using naturally occurring lactobacilli and yeast. The lactic acid produced by the lactobacilli imbues it a more sour taste, as well as extending its shelf life compared to other breads. Sourdough baking has a devoted community today. Many devotees share starters and tips via the Internet, and hobbyists often proudly share their work on social media.

Page: History of California bread
Summary: The history of California bread as a prominent factor in the field of bread baking dates from the days of the California Gold Ru

'Sourdough bread is a bread made by the fermentation of dough using wild lactobacillaceae and yeast. The fermentation process produces lactic acid, which gives the bread its characteristic sour taste and improves its keeping qualities. Sourdough bread has a devoted community of enthusiasts who share starters and tips online, and it has a rich history, particularly in San Francisco during the California Gold Rush. It is known for its artisanal qualities and is enjoyed by many bread lovers around the world.'

In [9]:
agent_executor.run("What have we talked about so far?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "We have talked about sourdough bread."
}

> Finished chain.


'We have talked about sourdough bread.'

In [10]:
memory.chat_memory.messages

[HumanMessage(content='What is sourdough bread?', additional_kwargs={}, example=False),
 AIMessage(content='Sourdough bread is a bread made by the fermentation of dough using wild lactobacillaceae and yeast. The fermentation process produces lactic acid, which gives the bread its characteristic sour taste and improves its keeping qualities. Sourdough bread has a devoted community of enthusiasts who share starters and tips online, and it has a rich history, particularly in San Francisco during the California Gold Rush. It is known for its artisanal qualities and is enjoyed by many bread lovers around the world.', additional_kwargs={}, example=False),
 HumanMessage(content='What have we talked about so far?', additional_kwargs={}, example=False),
 AIMessage(content='We have talked about sourdough bread.', additional_kwargs={}, example=False)]